In [ ]:
import os
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch
import torchvision.models as models
import torch.backends.cudnn as cudnn
cudnn.benchmark = True

In [ ]:
def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res


In [ ]:
class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self, name, fmt=':f'):
        self.name = name
        self.fmt = fmt
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

    def __str__(self):
        fmtstr = '{name} {val' + self.fmt + '} ({avg' + self.fmt + '})'
        return fmtstr.format(**self.__dict__)



In [ ]:
valdir = os.path.join('/mnt/disks/d1/s/data/apra/tempdatasets/june21/','val')
resume = 'resnet152models/checkpoint.pth.tar'
arch = 'resnet152'

In [ ]:
model = models.__dict__[arch]()
model = torch.nn.DataParallel(model).cuda()

In [ ]:
checkpoint = torch.load(resume)
model.load_state_dict(checkpoint['state_dict'])
# switch to evaluate mode
model.eval()

In [ ]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

val_dataset = datasets.ImageFolder(valdir, transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            normalize,
        ]))

val_loader = torch.utils.data.DataLoader(
        val_dataset,
        batch_size=256, shuffle=False,
        num_workers=4, pin_memory=True)


In [ ]:
top1 = AverageMeter('Acc@1', ':6.2f')
top5 = AverageMeter('Acc@5', ':6.2f') 

with torch.no_grad():        
        for i, (images, target) in enumerate(val_loader):
            target = target.cuda(None, non_blocking=True)

            # compute output
            output = model(images)
            
            # measure accuracy and record loss
            acc1, acc5 = accuracy(output, target, topk=(1, 5))
            top1.update(acc1[0], images.size(0))
            top5.update(acc5[0], images.size(0))

            
            print("processed<", i, "> output<", output.size(), '>')

        # TODO: this should also be done with the ProgressMeter
        print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))
print(' * Acc@1 {top1.avg:.3f} Acc@5 {top5.avg:.3f}'
              .format(top1=top1, top5=top5))

In [ ]:
_, pred = output.topk(1, 1, True, True)

In [ ]:
import csv
csvfile = open("results.csv", 'w')
csvwriter = csv.writer(csvfile)
# csvwriter.writerows(pred.cpu().numpy())
csvwriter.writerows(target.view(1, -1).expand_as(pred.t()).t().cpu().numpy())
csvfile.close()

In [ ]:
pred.cpu().t().numpy()

In [ ]:
target.size()

In [ ]:
target.view(1, -1).expand_as(pred.t()).cpu().numpy()

In [ ]:
target.cpu().numpy()